#### Import libs

In [2]:
#base
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as numpy
#torch
import torch
from torch import nn

import warnings
warnings.filterwarnings('ignore')

#transformers
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
model_checkpoint = 'cointegrated/rubert-tiny-toxicity'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

In [9]:
df = pd.read_csv('/home/marena/Elbrus_phase_2/nlp_project/data/labeled.csv')

In [10]:
df.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [11]:
y = df['toxic']
X = df['comment']

In [14]:
encoded_y = y.apply(
    lambda x: bool(x)
)
encoded_y.value_counts()

toxic
False    9586
True     4826
Name: count, dtype: int64

In [15]:
def text2toxicity(text, aggregate):
    """ Calculate toxicity of a text (if aggregate=True) or a vector of toxicity aspects (if aggregate=False)"""
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()
    if isinstance(text, str):
        proba = proba[0]
    if aggregate:
        return 1 - proba.T[0] * (1 - proba.T[-1])
    return proba

In [16]:
text2toxicity(X[0], encoded_y[0])

0.9955001367198989